In [12]:
import openai
import os
import json
import xml.etree.ElementTree as ET

# os.environ["http_proxy"] = "http://127.0.0.1:7890"
# os.environ["https_proxy"] = "http://127.0.0.1:7890"

openai.api_key="your key"


MODELING_PROMPT='''
You are designed to be a professional GEE developer and geospatial modeler, and generate model using ArcPy following my guidance:

Please think step by step:
{}

Please follow the instructions and do not forget them:
(1) You should understand the provided modeling process and involved functions, refer to raw code, and then generate specific code using ArcPy;
(2) Generate your modeling plan first in the [Modeling Plan] block exactly;
(3) Thenprovide python code in ```python``` block;
(4) Do not miss code for any step in the python code;
(5) The python code should be execuable;

'''


def extract_info_from_xml(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract task name from the root element's attributes
    template_name = root.get('name', 'Unknown Task')

    # Extract task description from the Task element
    task_value = root.find('.//Task/Value')
    task_name = task_value.text if task_value is not None else 'Unknown Task'

    # Extract place name from GeospatialRange
    geospatial_range = root.find('.//GeospatialRange/Value')
    place_name = geospatial_range.text if geospatial_range is not None else 'Unknown Place'

    # Extract datasources
    datasources = []
    for source in root.findall('.//Datasource/Source'):
        source_name = source.find('SourceName').text if source.find('SourceName') is not None else 'Unknown Source'
        datasources.append(source_name)

    # Extract process steps from ServiceNode and Relationship elements
    service_nodes = {}
    for node in root.findall('.//ServiceNode/Node'):
        node_id = node.get('id', 'Unknown ID')
        node_type = node.get('type', 'Unknown Type')
        node_value = node.text if node.text is not None else 'Unknown Value'
        service_nodes[node_id] = node_value

    service_nodes_descriptions = []
    for node_id, node_value in service_nodes.items():
        node_type = root.find(f".//ServiceNode/Node[@id='{node_id}']").get('type', 'Unknown Type')
        service_nodes_descriptions.append(f"This model contains a node ({node_value}) with type {node_type}")

    # Extract and organize relationships
    relationships = {}
    for relation in root.findall('.//Relationship/Relation'):
        target_node_id = relation.get('targetNodeId', 'Unknown Target ID')
        source_node_id = relation.get('sourceNodeId', 'Unknown Source ID')
        if target_node_id not in relationships:
            relationships[target_node_id] = []
        relationships[target_node_id].append(source_node_id)

    relationships_descriptions = []
    for target_node_id, source_node_ids in relationships.items():
        target_value = service_nodes.get(target_node_id, 'Unknown Target Value')
        source_values = [service_nodes.get(source_id, 'Unknown Source Value') for source_id in source_node_ids]
        relationships_descriptions.append(f"Node ({target_value}) is connected to nodes ({', '.join(source_values)})")

    # Extract raw code from SourceCode
    raw_code = []
    for content in root.findall('.//SourceCode/Script/Content'):
        raw_code.append(content.text.strip() if content.text is not None else '')

    # Generate the description
    description = f"""(1) Generate a geographic analysis model for the task: {task_name+';'+template_name};
(2) The model is performed in the place: {place_name};
(3) The model employs datasource: {', '.join(datasources)};
(4) Relative modeling process in Google Earth Engine is described **structurally** as follow:
```Service Nodes```
{chr(10).join(service_nodes_descriptions)}

```Relationships```
{chr(10).join(relationships_descriptions)}

'''Raw Code'''
{chr(10).join(raw_code)}
"""
    return description

# Example usage
xml_file_path = 'landslide_susceptility_evaluation.xml'
description = extract_info_from_xml(xml_file_path)


def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.2, messages=None):
    '''
    Retrieve completions from the OpenAI API, preserving conversation history.
    '''
    if messages is None:
        messages = []

    messages.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    response_content = response.choices[0].message.content
    messages.append({"role": "system", "content": response_content})
    return response_content, messages



reasoning_prompt = MODELING_PROMPT.format(description)

print(f'[REASONING PROMPT]:\n{reasoning_prompt}')
reasoning_response, _ = get_completion(reasoning_prompt,model="gpt-3.5-turbo",temperature=0.2)
print("="*150)
print(f'[RESPONSE]:\n{reasoning_response}')


[REASONING PROMPT]:

You are designed to be a professional GEE developer and geospatial modeler, and generate model using ArcPy following my guidance:

Please think step by step:
(1) Generate a geographic analysis model for the task: Land surface;landslide_susceptility_evaluation;
(2) The model is performed in the place: Xunwu Xian(geonameId=1787876);
(3) The model employs datasource: users/region/XunwuCounty/Xunwu, users/region/XunwuCounty/Landsat_2018, NASA SRTM Digital Elevation 30m;
(4) Relative modeling process in Google Earth Engine is described **structurally** as follow:
```Service Nodes```
This model contains a node (USGS/SRTMGL1_003) with type literal
This model contains a node (ee.Image) with type eeFunction
This model contains a node (ee.Terrain.slope) with type eeFunction
This model contains a node (expression) with type eeFunction
This model contains a node (B5) with type literal
This model contains a node (ee.List) with type eeFunction
This model contains a node (B4) wit